In [1]:
import gymnasium as gym
import numpy as np
from mujoco import viewer
from typing import Union

from gymnasium_robotics.envs.robot_env import MujocoPyRobotEnv, MujocoRobotEnv
from gymnasium_robotics.utils import rotations


/Users/cth/miniforge3/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FetchSlide-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/cth/miniforge3/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FetchSlide-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/cth/miniforge3/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FetchPickAndPlace-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/cth/miniforge3/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FetchPickAndPlace-v3 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/Users/cth/miniforge3/lib/

In [2]:

# Define three sets of camera parameters
camera_params_front = {
    "azimuth": 0.0,
    "elevation": -14.0,
    "distance": 2.5,
    "lookat": np.array([1.3, 0.75, 0.55])
}

camera_params_right = {
    "azimuth": 60.0,
    "elevation": -14.0,
    "distance": 2.5,
    "lookat": np.array([1.3, 0.75, 0.55])
}

camera_params_left = {
    "azimuth": -60.0,
    "elevation": -14.0,
    "distance": 2.5,
    "lookat": np.array([1.3, 0.75, 0.55])
}




In [3]:

# Test the camera method by Camera ID (integer)
try:
    camera_views_by_id = env.unwrapped.data.camera(1)  # Assuming 0 is a valid camera ID
    print("Camera views by ID:")
    print(camera_views_by_id)
except Exception as e:
    print(f"Error using camera ID: {e}")

# Test the camera method by Camera Name (string)
try:
    camera_views_by_name = env.unwrapped.data.camera('0')  # Adjust the name as necessary
    print("Camera views by Name:")
    print(camera_views_by_name)
except Exception as e:
    print(f"Error using camera name: {e}")

try:
    camera_data = env.unwrapped.data.cam('external_camera_0')  # Adjust the name as necessary
    print("data of cam:")
    print(camera_data)
except Exception as e:
    print(f"Error using cam index: {e}")



Error using camera ID: name 'env' is not defined
Error using camera name: name 'env' is not defined
Error using cam index: name 'env' is not defined


In [4]:
import numpy as np

def azimuth_elevation_to_matrix(azimuth, elevation):
    """Convert azimuth and elevation to a rotation matrix (xmat)."""
    # Convert degrees to radians
    azimuth = np.radians(azimuth)
    elevation = np.radians(elevation)

    # Compute rotation matrices around the Z-axis (azimuth) and X-axis (elevation)
    Rz = np.array([
        [np.cos(azimuth), -np.sin(azimuth), 0],
        [np.sin(azimuth), np.cos(azimuth), 0],
        [0, 0, 1]
    ])
    
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(elevation), -np.sin(elevation)],
        [0, np.sin(elevation), np.cos(elevation)]
    ])

    # Combine the rotations
    R = np.dot(Rz, Rx)

    # Flatten the matrix to a 9-element array
    xmat = R.flatten()
    return xmat

def setup_camera_parameters(params, camera_name):
    """Convert camera parameters to the desired format."""
    azimuth = params["azimuth"]
    elevation = params["elevation"]
    distance = params["distance"]
    lookat = params["lookat"]
    
    # Compute the rotation matrix
    xmat = azimuth_elevation_to_matrix(azimuth, elevation)
    
    # Compute the camera position (xpos) based on distance and lookat point
    direction = np.array([0, 0, 1])  # Assuming a default direction along the Z-axis
    direction_rotated = np.dot(xmat.reshape((3, 3)), direction)
    xpos = lookat - distance * direction_rotated

    camera_config = {
        "name": camera_name,
        "xmat": xmat,
        "xpos": xpos
    }
    
    return camera_config

# Example camera parameters
camera_params_front = {
    "azimuth": 0.0,
    "elevation": -14.0,
    "distance": 2.5,
    "lookat": np.array([1.3, 0.75, 0.55])
}

camera_params_right = {
    "azimuth": 60.0,
    "elevation": -14.0,
    "distance": 2.5,
    "lookat": np.array([1.3, 0.75, 0.55])
}

camera_params_left = {
    "azimuth": -60.0,
    "elevation": -14.0,
    "distance": 2.5,
    "lookat": np.array([1.3, 0.75, 0.55])
}

# Convert the camera parameters
camera_config_front = setup_camera_parameters(camera_params_front, "external_camera_0")
camera_config_right = setup_camera_parameters(camera_params_right, "external_camera_0")
camera_config_left = setup_camera_parameters(camera_params_left, "external_camera_0")

print(camera_config_front)
print(camera_config_right)
print(camera_config_left)


{'name': 'external_camera_0', 'xmat': array([ 1.        ,  0.        ,  0.        ,  0.        ,  0.97029573,
        0.2419219 ,  0.        , -0.2419219 ,  0.97029573]), 'xpos': array([ 1.3       ,  0.14519526, -1.87573932])}
{'name': 'external_camera_0', 'xmat': array([ 0.5       , -0.84030075, -0.20951051,  0.8660254 ,  0.48514786,
        0.12096095,  0.        , -0.2419219 ,  0.97029573]), 'xpos': array([ 1.82377627,  0.44759763, -1.87573932])}
{'name': 'external_camera_0', 'xmat': array([ 0.5       ,  0.84030075,  0.20951051, -0.8660254 ,  0.48514786,
        0.12096095,  0.        , -0.2419219 ,  0.97029573]), 'xpos': array([ 0.77622373,  0.44759763, -1.87573932])}


In [5]:

# Define the function to set up the environment with different camera viewpoints
def setup_env_with_viewpoint(env_id, render_mode="rgb_array", camera_config=None):

    # Initialize the environment
    env = gym.make(env_id, render_mode=render_mode)
    env.reset()
    env.render()  # Render once to initialize the viewer
    
    
    env.unwrapped.data.cam_xpos[0] = camera_config['xpos']
    env.unwrapped.data.cam_xmat[0] = camera_config['xmat']
    
    return env

# Define a simple random policy for demonstration
def random_policy(env):
    return env.action_space.sample()

In [6]:
# import matplotlib.pyplot as plt
# # Set up Matplotlib figure



# # Set up and run the environment with the right camera parameters
# env_id = "FetchPickAndPlace-v3"

# # Initialize the environment with the right camera parameters
# env = setup_env_with_viewpoint(env_id, camera_config=camera_config_front)

# # Reset the environment to get the initial observation
# observation, info = env.reset(seed=42)

# env.reset()

# plt.ion()
# fig, ax = plt.subplots()

# # Run the simulation for 1000 steps with the current camera view
# for _ in range(1000):
#     # print("Current camera parameters:", env.unwrapped.data.cam_xpos, env.unwrapped.data.cam_xmat)
#     action = random_policy(env)  # Use the random policy to generate actions
#     observation, reward, terminated, truncated, info = env.step(action)

#     # Render the environment and display it using Matplotlib
#     rgb = env.render()
#     ax.clear()
#     ax.imshow(rgb)
#     ax.set_title("Current View")
#     plt.pause(0.01)

#     # Check if the episode is terminated or truncated
#     if terminated or truncated:
#         observation, info = env.reset()

# # Close the environment to release resources
# env.close()
# plt.ioff()
# plt.show()

In [7]:
import gymnasium as gym
import numpy as np
import cv2

def azimuth_elevation_to_matrix(azimuth, elevation):
    """Convert azimuth and elevation to a rotation matrix (xmat)."""
    # Convert degrees to radians
    azimuth = np.radians(azimuth)
    elevation = np.radians(elevation)

    # Compute rotation matrices around the Z-axis (azimuth) and X-axis (elevation)
    Rz = np.array([
        [np.cos(azimuth), -np.sin(azimuth), 0],
        [np.sin(azimuth), np.cos(azimuth), 0],
        [0, 0, 1]
    ])
    
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(elevation), -np.sin(elevation)],
        [0, np.sin(elevation), np.cos(elevation)]
    ])

    # Combine the rotations
    R = np.dot(Rz, Rx)

    # Flatten the matrix to a 9-element array
    xmat = R.flatten()
    return xmat

def setup_camera_parameters(params):
    """Convert camera parameters to the desired format."""
    azimuth = params["azimuth"]
    elevation = params["elevation"]
    distance = params["distance"]
    lookat = params["lookat"]
    
    # Compute the rotation matrix
    xmat = azimuth_elevation_to_matrix(azimuth, elevation)
    
    # Compute the camera position (xpos) based on distance and lookat point
    direction = np.array([0, 0, 1])  # Assuming a default direction along the Z-axis
    direction_rotated = np.dot(xmat.reshape((3, 3)), direction)
    xpos = lookat - distance * direction_rotated

    camera_config = {
        "xmat": xmat,
        "xpos": xpos
    }
    
    return camera_config

def setup_env_with_viewpoint(env, camera_config=None):
    """Update the environment's camera configuration."""
    env.unwrapped.data.cam_xpos[0] = camera_config['xpos']  # Assuming the first camera
    env.unwrapped.data.cam_xmat[0] = camera_config['xmat']

# Define a simple random policy for demonstration
def random_policy(env):
    return env.action_space.sample()

# Define three sets of camera parameters
camera_params_right = {
    "azimuth": 60.0,
    "elevation": -14.0,
    "distance": 2.5,
    "lookat": np.array([1.3, 0.75, 0.55])
}

camera_params_left = {
    "azimuth": -60.0,
    "elevation": -14.0,
    "distance": 2.5,
    "lookat": np.array([1.3, 0.75, 0.55])
}

# Convert the camera parameters
camera_config_right = setup_camera_parameters(camera_params_right)
camera_config_left = setup_camera_parameters(camera_params_left)

# Set up and run the environment with the right camera parameters
env_id = "FetchPickAndPlace-v3"

# Initialize the environment
env = gym.make(env_id, render_mode="rgb_array")
env.reset()

use_right_camera = True
# Initial camera configuration
current_camera_config = camera_config_right

# Run the simulation for 1000 steps with alternating camera views
for step in range(10000):
    if step % 100 == 0:
        # Switch camera configuration every 100 steps
        if use_right_camera:
            current_camera_config = camera_config_left
        else:
            current_camera_config = camera_config_right
        
        use_right_camera = not use_right_camera  # Toggle the flag
       
        
        # Update the camera configuration in the environment
        setup_env_with_viewpoint(env, current_camera_config)

    print("Step:", step, "Current camera parameters:", env.unwrapped.data.cam_xpos[0], env.unwrapped.data.cam_xmat[0])
    action = random_policy(env)  # Use the random policy to generate actions
    observation, reward, terminated, truncated, info = env.step(action)

    # Render the environment and display it using OpenCV
    rgb = env.render()
    rgb = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    cv2.imshow("Current View", rgb)
    cv2.waitKey(30)

    # Check if the episode is terminated or truncated
    if terminated or truncated:
        observation, info = env.reset()

# Close the environment to release resources
env.close()
cv2.destroyAllWindows()


Step: 0 Current camera parameters: [ 0.77622373  0.44759763 -1.87573932] [ 0.5         0.84030075  0.20951051 -0.8660254   0.48514786  0.12096095
  0.         -0.2419219   0.97029573]
Step: 1 Current camera parameters: [0.45060007 0.2841     1.06098698] [ 3.63910180e-10  9.54947876e-05 -9.99999995e-01 -1.00000000e+00
  3.48054918e-14 -3.63910124e-10  5.55111512e-17  9.99999995e-01
  9.54947876e-05]
Step: 2 Current camera parameters: [0.45060016 0.2841     1.06117842] [ 8.70604977e-10  9.83991733e-05 -9.99999995e-01 -1.00000000e+00
  8.56537063e-14 -8.70604921e-10  0.00000000e+00  9.99999995e-01
  9.83991733e-05]
Step: 3 Current camera parameters: [0.45060027 0.2841     1.06142176] [ 1.07897935e-09  1.01292570e-04 -9.99999995e-01 -1.00000000e+00
  1.09356968e-13 -1.07897946e-09  5.55111512e-17  9.99999995e-01
  1.01292570e-04]
Step: 4 Current camera parameters: [0.45060037 0.2841     1.06165381] [ 9.80948323e-10  1.04174645e-04 -9.99999995e-01 -1.00000000e+00
  1.01862963e-13 -9.8094826

KeyboardInterrupt: 